In [1]:
import pandas as pd

In [3]:
reload(userinput)

<module 'userinput' from 'userinput.pyc'>

In [4]:
import userinput
fn = r"/Users/dblyon/Downloads/Data for agotool 2017-01-30.txt"
import db_config
import io
import tools
ECHO = False
TESTING = False
DO_LOGGING = False
connection = db_config.Connect(echo=ECHO, testing=TESTING, do_logging=DO_LOGGING)


fn = io.open(fn, mode='r')
ui = userinput.Userinput(fn=fn, foreground_string=None,
    background_string=None, col_foreground='foreground',
    col_background='background', col_intensity='intensity',
    num_bins=100, decimal='.', method="abundance_correction", connection=connection)

In [5]:
ans_list = ui.get_all_unique_ANs()
len(ans_list)

2375

In [7]:
def reduce_UniProtKeywordsALL_2_ans_list(fn_keywords, ans_list, fn_out):
    ans_set = set(ans_list)
    with open(fn_keywords, "r") as fh_in:
        with open(fn_out, "w") as fh_out:
            header = "Entry\tKeywords\n"
            fh_out.write(header)
            for line in fh_in:
                an = line.split("\t")[0]
                if an in ans_set:
                    fh_out.write(line)

In [8]:
fn_keywords = r"/Users/dblyon/Downloads/uniprot-all.tab"
fn_out = r"/Users/dblyon/modules/cpr/metaprot/sql/downloads/keywords_temp.upk"
reduce_UniProtKeywordsALL_2_ans_list(fn_keywords, ans_list, fn_out)

In [10]:
ui.background_n

In [15]:
ui.get_background_n(), ui.get_foreground_n()

(1803, 1803)

In [18]:
len(ui.foreground["foreground"].unique())

1803

In [7]:
DEFAULT_MISSING_BIN = -1
cond = ui.foreground["intensity"] > DEFAULT_MISSING_BIN
bins = pd.cut(ui.foreground.loc[cond, "intensity"], bins=ui.num_bins, retbins=True, include_lowest=True)[1]
print bins[:3], bins[-3:] # [ -2.82264587e+07   2.84369436e+08   5.68547523e+08] [  2.78496438e+10   2.81338219e+10   2.84180000e+10]
# bins = [DEFAULT_MISSING_BIN + 1] + list(bins)
# print(self.foreground.head())
# groups_fg = self.foreground.groupby(pd.cut(self.foreground["intensity"], bins=bins))
# groups_bg = self.background.groupby(pd.cut(self.background[self.col_intensity], bins=bins))

[ -2.82264587e+07   2.84369436e+08   5.68547523e+08] [  2.78496438e+10   2.81338219e+10   2.84180000e+10]


In [8]:
int(-2.82264587e+07)

-28226458

In [9]:
groups_fg = ui.foreground.groupby(pd.cut(ui.foreground["intensity"], bins=bins))

In [48]:
ui.foreground.sort_values("intensity", ascending=True, inplace=True)

In [50]:
ui.foreground["intensity"].max()

28418000000.0

In [11]:
import tools
gocat_upk, function_type, limit_2_parent = "UPK", "UPK", None

In [17]:
# protein_ans_list = ui.get_all_unique_ANs()
protein_ans_list = ["P00359", "P38701"]

In [18]:
go_slim_or_basic = "basic"
backtracking = True
assoc_dict = tools.get_association_dict(connection, protein_ans_list, function_type, limit_2_parent=limit_2_parent, basic_or_slim=go_slim_or_basic, backtracking=backtracking)

In [19]:
assoc_dict

defaultdict(<function tools.<lambda>>,
            {u'P00359': {u'UPK:0002',
              u'UPK:0181',
              u'UPK:0324',
              u'UPK:0560',
              u'UPK:0597',
              u'UPK:0832',
              u'UPK:0903',
              u'UPK:0963',
              u'UPK:1017',
              u'UPK:1185',
              u'UPK:9990',
              u'UPK:9991',
              u'UPK:9992',
              u'UPK:9993',
              u'UPK:9998',
              u'UPK:9999'},
             u'P38701': {u'UPK:0002',
              u'UPK:0181',
              u'UPK:0687',
              u'UPK:0689',
              u'UPK:0832',
              u'UPK:0903',
              u'UPK:0963',
              u'UPK:1017',
              u'UPK:1185',
              u'UPK:9990',
              u'UPK:9991',
              u'UPK:9992',
              u'UPK:9998'}})

In [26]:
import requests

In [27]:
def download_file(url, fn_out):
    """
    only works for http not ftp
    """
    r = requests.get(url, stream=True)
    with open(fn_out, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

In [28]:
import os
url = r"http://www.uniprot.org/uniprot/?columns=id,keywords&format=tab"
DOWNLOADS_DIR = r"/Users/dblyon/modules/cpr/metaprot/sql/downloads"
file_name = os.path.join(DOWNLOADS_DIR, 'ALL.upk')
tmp_f = os.path.join(DOWNLOADS_DIR, 'keywords.tmp')
print('%s\nDownloaded to: %s\n' % (url, file_name))
download_file(url, tmp_f)
os.rename(tmp_f, file_name)

http://www.uniprot.org/uniprot/?columns=id,keywords&format=tab
Downloaded to: /Users/dblyon/modules/cpr/metaprot/sql/downloads/ALL.upk



In [ ]:
http://www.uniprot.org/uniprot/?query=9606:%i&columns=id,keywords_id&format=tab

In [ ]:
#!/usr/local/bin/python
python /Users/dblyon/modules/cpr/metaprot/sql/create_SQL_tables.py
python /Users/dblyon/modules/cpr/metaprot/sql/models.py